In [1]:
# coding: utf-8
import requests
import csv
from bs4 import BeautifulSoup

def get_the_page(url): #get one page's restaurants info
    
    r = requests.get(url,headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36'})
    html_str = r.text
    mypage = BeautifulSoup(html_str,"html.parser")
    
    myitems = mypage.find_all('li',attrs={'class':'sr1-listing-content-cell pois-restaurant-list-cell '})
    
    titles=[]
    likes=[]
    locations=[]
    prices=[]
    countries=[]
    styles=[]
    reviews=[]
    bookmarks=[]
    discounts=[]
    
    for myitem in myitems:
        
        titles.append(myitem.find('h2').text.strip())   #find titles
             
        like=myitem.find('span',attrs={'class':'score score-big highlight'})   #find likes
        if like is not None:       #some item doesn't have like or other features
            likes.append(like.text)
        else:
            like = '0'
            likes.append(like)
        
        location = myitem.find('div',attrs={'class':'icon-info address'})   #find locations
        if location is not None:      
            locations.append(location.find('span').text.strip())
        else:
            location = 'NONE'
            locations.append(location)
        
        price = myitem.find('div',attrs={'class': 'icon-info icon-info-food-price'})   #find prices
        if price is not None:       
            prices.append(price.find('span').text)
        else:
            price = 'NONE'
            prices.append(price)
        
        review = myitem.find('div',attrs={'class':'counters-container'})  #dind review numbers
        if review is not None:       
            reviews.append(review.find('span').text)
        else:
            review = '0'
            reviews.append(review)
        
        bookmark = myitem.find('div',attrs={'class':'text bookmarkedUserCount js-bookmark-count'})  #find bookmark
        if bookmark is not None:       
            bookmarks.append(bookmark['data-count'])
        else:
            bookmark = '0'
            bookmarks.append(bookmark)

        discount = myitem.find('div',attrs={'class':'info-row'})  #find discount info
        if discount is not None:       
            discounts.append(discount.find('span').text)
        else:
            discount = 'NONE'
            discounts.append(discount)
              
        countries_material=myitem.find("ul",attrs={'class':'pois-categoryui-list'}).find_all('li')

        countries.append(countries_material[0].text.strip())     #find country
        if len(countries_material)>1:
            styles.append(countries_material[1].text.strip())    #find likes
        else:
            styles.append(countries_material[0].text.strip())
    
    print(url)
    
    with open('openrice_sample.csv', 'a') as f:
        writer = csv.writer(f)
        for i in range(0,len(titles)):
            writer.writerow([titles[i],locations[i],prices[i],countries[i],styles[i],likes[i],reviews[i],bookmarks[i],discounts[i]])
    return

In [2]:
def scrape_pages(url_page1):    #get 17 pages, which is limit of open rice, of the results return from your search
    get_the_page(url_page1)
    for q in range(1,17):
        page_number=str(q+1)
        url_next='{0}&page={1}'.format(url_page1,page_number)
        get_the_page(url_next)
    return

In [3]:
url_list = ['https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-09-30&BookingPeriod=Breakfast',
            'https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-09-30&BookingPeriod=Lunch',
            'https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-09-30&BookingPeriod=Tea',
            'https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-09-30&BookingPeriod=Dinner']

In [4]:
fail = []
for url in url_list:
    try: 
        scrape_pages(url)
    except Exception as e:
        print(e)
        print(url)
        fail.append((url, e))

https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-09-30&BookingPeriod=Breakfast
https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-09-30&BookingPeriod=Breakfast&page=2
https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-09-30&BookingPeriod=Breakfast&page=3
https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-09-30&BookingPeriod=Breakfast&page=4
https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-09-30&BookingPeriod=Breakfast&page=5
https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-09-30&BookingPeriod=Breakfast&page=6
https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-09-30&BookingPeriod=Breakfast&page=7
https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-09-30&BookingPeriod=Breakfast&page=8
https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-09-30&BookingPeriod=Breakfast&page=9
https://www.openr

In [5]:
import pandas
df = pandas.read_csv('openrice_sample.csv',header=None, names=['name', 'location', 'price', 'country', 'style', 'likes','review','bookmark','discount_info']) 

In [6]:
len(df)

932

In [7]:
df = df.drop_duplicates() #remove duplicated rows

In [8]:
len(df) #i guess its because some restaurants are opening all day

252

In [9]:
df

,name,location,price,country,style,likes,review,bookmark,discount_info
0,LAB EAT Restaurant & Bar,尖沙咀金巴利道87-89號僑豐大厦地下1-2號舖,$201-400,西式,海鮮,436,(565 食評),50655,網上訂座可享75折優惠
1,Shine,尖沙咀北京道12A號太子集團中心6樓,$201-400,西式,酒,692,(776 食評),21060,送 25里數 / 30積分
2,Espuma,尖沙咀厚福街8號H8 2樓,$101-200,西班牙菜,甜品/糖水,610,(949 食評),40370,送 25里數 / 30積分
3,The Captain's House,尖沙咀厚福街8號H8 18樓,$201-400,西式,海鮮,511,(634 食評),33129,送 25里數 / 30積分
4,Yadllie Plate,旺角西洋菜街1號兆萬中心11樓,$101-200,韓國菜,韓式炸雞,487,(630 食評),27531,早鳥及消夜時段訂座 可享全單八折優惠
5,Day and Nite by Master Kama,旺角山東街50號1-2樓,$101-200,日本菜,海鮮,462,(595 食評),28138,送 25里數 / 30積分
6,漁獲浜燒 Toretore Hamayaki,銅鑼灣軒尼詩道525號澳門逸園中心18樓,$201-400,日本菜,日本菜,503,(524 食評),19623,送 25里數 / 30積分
7,The Grill Room,銅鑼灣駱克道459-461號The L. Square 5樓,$201-400,西式,扒房,508,(662 食評),32282,訂座驚喜星期六及日: 套餐$238起-全日酒精飲品Happy Hour 價錢
8,心之食堂 Love Cafe,銅鑼灣耀華街3號百樂中心12樓1203室,$201-400,日本菜,壽司/刺身,487,(542 食評),6541,送 25里數 / 30積分
9,Sky726,旺角彌敦道724-726號25樓,$201-400,法國菜,甜品/糖水,367,(494 食評),38427,自選餐牌主菜買一送一優惠 或 甜品8折優惠； 飲品一律8折


In [10]:
df.to_csv('openrice_sample.csv', encoding='utf-8', index=False)